# SETTING ENVIRONMENT


In [1]:
"""# mount the colab with google drive
from google.colab import drive
drive.mount('/content/drive')"""

"# mount the colab with google drive\nfrom google.colab import drive\ndrive.mount('/content/drive')"

In [2]:
# set folder tempat kerja (current working directory)
import os
cwd = '/Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita/monitoring-berita'

#cwd = '/content/drive/MyDrive/Monitoring Berita'
os.chdir(cwd)

# MAIN CODE

In [3]:
# === Import & Setup ===
import os
import re
import logging
from pathlib import Path

import pandas as pd
from tqdm.auto import tqdm

# (Opsional) Jika di Google Colab, pastikan Drive sudah di-mount
# from google.colab import drive
# drive.mount('/content/drive')

# Logging yang rapi
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
)

# Folder akar yang akan dipindai (disesuaikan ke workspace lokal)
# Gunakan folder 'daftar_berita' yang sekarang berisi SEMUA file xlsx (tanpa subfolder per portal)
ROOT_DIR = Path(cwd) / "daftar_berita"

# Kolom target (akan dipastikan tersedia setelah normalisasi)
TARGET_COLS = ["judul_berita", "tanggal_berita", "url_berita"]

# Mapping normalisasi nama kolom
COLUMN_ALIASES = {
    "judulberita": "judul_berita",
    "judul_berita": "judul_berita",
    "judul": "judul_berita",

    "tanggalberita": "tanggal_berita",
    "tanggal_berita": "tanggal_berita",
    "tanggal": "tanggal_berita",
    "tgl": "tanggal_berita",
    "date": "tanggal_berita",

    "penulisberita": "penulis_berita",
    "penulis_berita": "penulis_berita",
    "penulis": "penulis_berita",
    "author": "penulis_berita",

    "urlberita": "url_berita",
    "url_berita": "url_berita",
    "url": "url_berita",
    "link": "url_berita",
}

In [4]:
def normalize_col(col: str) -> str:
    """Normalisasi nama kolom: lowercase + remove non-alnum."""
    if col is None:
        return ""
    c = str(col).strip().lower()
    c = re.sub(r"[^a-z0-9]+", "", c)
    return c

def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Standarkan nama kolom dataframe sesuai TARGET_COLS memakai COLUMN_ALIASES.
    """
    if df is None or df.empty:
        return pd.DataFrame(columns=TARGET_COLS)

    rename_map = {}
    for col in df.columns:
        norm = normalize_col(col)
        if norm in COLUMN_ALIASES:
            rename_map[col] = COLUMN_ALIASES[norm]

    df2 = df.rename(columns=rename_map)

    # Pastikan semua kolom target ada
    for c in TARGET_COLS:
        if c not in df2.columns:
            df2[c] = pd.NA

    return df2[TARGET_COLS]

def read_excel_safe(path: Path) -> pd.DataFrame:
    """
    Membaca satu file Excel dengan fallback engine dan error handling.
    """
    try:
        df = pd.read_excel(path)
    except Exception as e1:
        try:
            if path.suffix.lower() in [".xlsx", ".xlsm", ".xltx", ".xltm"]:
                df = pd.read_excel(path, engine="openpyxl")
            elif path.suffix.lower() == ".xls":
                df = pd.read_excel(path, engine=None)
            else:
                raise
        except Exception as e2:
            logging.warning(f"Gagal membaca: {path} | err1={e1} | err2={e2}")
            return pd.DataFrame(columns=TARGET_COLS)

    df = standardize_columns(df).dropna(how="all")
    df = df[~(df["judul_berita"].isna() & df["url_berita"].isna())]
    return df


In [5]:
# Cari semua file Excel secara rekursif (struktur baru: semua file xlsx ada langsung di folder ini)
excel_patterns = ["*.xlsx", "*.xls", "*.xlsm", "*.xltx", "*.xltm"]
paths = []
for pat in excel_patterns:
    # gunakan glob (bukan rglob) untuk satu level, namun tetap dukung rglob kalau ada subfolder tersisa
    paths.extend(sorted(ROOT_DIR.glob(pat)))
    paths.extend(sorted(ROOT_DIR.rglob(pat)))
# unikkan path
paths = sorted(dict.fromkeys(paths))

if not paths:
    logging.warning(f"Tidak ditemukan file Excel di bawah {ROOT_DIR}")

dfs = []
for p in tqdm(paths, desc="Membaca Excel", unit="file"):
    df_part = read_excel_safe(p)
    if not df_part.empty:
        # Coba tebak nama portal dari nama file jika tidak ada subfolder
        # Contoh: kompas.xlsx -> kompas, cnn_links.xlsx -> cnn
        stem = p.stem.lower()
        media_guess = re.split(r"[_\-]", stem)[0] if stem else "unknown"
        media_online = p.parent.name if p.parent != ROOT_DIR else media_guess

        # Tambahkan metadata
        df_part = df_part.assign(
            media_online=media_online,
            _source=str(p)
        )
        dfs.append(df_part)

# Gabungkan semua
if dfs:
    df_all = pd.concat(dfs, ignore_index=True)
else:
    df_all = pd.DataFrame(columns=TARGET_COLS + ["media_online", "_source"])

# --- Drop duplikat berdasarkan url_berita ---
if "url_berita" in df_all.columns:
    before = len(df_all)
    df_all = df_all.drop_duplicates(subset=["url_berita"], keep="first")
    after = len(df_all)
    logging.info(f"Duplikat dihapus: {before - after} baris | Total akhir: {after}")

# Strip whitespace pada teks
for c in ["judul_berita", "penulis_berita", "url_berita", "tanggal_berita"]:
    if c in df_all.columns:
        df_all[c] = df_all[c].astype("string").str.strip()

logging.info(f"Selesai gabung. Total baris: {len(df_all)} | File terbaca: {len(dfs)}")

#display(df_all.head(10))

Membaca Excel:   0%|          | 0/9 [00:00<?, ?file/s]

2025-09-28 16:47:36,290 | INFO | Duplikat dihapus: 40 baris | Total akhir: 575
2025-09-28 16:47:36,292 | INFO | Selesai gabung. Total baris: 575 | File terbaca: 9


In [6]:
import re
import logging
from datetime import datetime
from zoneinfo import ZoneInfo

import pandas as pd
from dateutil import parser as du_parser

# =========================
# Setup logging (opsional)
# =========================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
)

# =======================================================
# Normalisasi kolom tanggal berita -> ISO Asia/Jakarta
# =======================================================
def normalize_tanggal_berita(
    df: pd.DataFrame,
    col_candidates=("tanggal_berita", "tanggal berita"),
    target_tz:str="Asia/Jakarta",
    output_col_str:str="tanggal_berita_std",
    output_col_dt:str="tanggal_berita_dt",
    error_col:str="tanggal_parse_error",
) -> pd.DataFrame:
    """
    Menormalkan berbagai format tanggal hasil scraping ke ISO 8601 +07:00.

    - Mencari kolom tanggal berdasarkan urutan `col_candidates`.
    - Menghapus nama hari (Senin..Minggu), koma, dan penanda WIB/WITA/WIT.
    - Mengonversi nama bulan Indonesia (pendek & panjang).
    - Jika datetime tanpa timezone -> dilokalkan ke Asia/Jakarta.
    - Jika punya timezone -> dikonversi ke Asia/Jakarta.
    - Tanggal tanpa jam -> diasumsikan 00:00:00 di Asia/Jakarta.
    - Menambahkan:
        * `tanggal_berita_std` (string ISO 8601)
        * `tanggal_berita_dt` (datetime timezone-aware)
        * `tanggal_parse_error` (string error jika ada, else None)
    """
    # Tentukan kolom sumber
    src_col = None
    for c in col_candidates:
        if c in df.columns:
            src_col = c
            break
    if src_col is None:
        logging.warning(f"Tidak menemukan kolom tanggal. Coba salah satu: {col_candidates}")
        # tidak fatal, kembalikan df apa adanya
        return df

    tz_map = {
        "WIB": ZoneInfo("Asia/Jakarta"),      # UTC+7
        "WITA": ZoneInfo("Asia/Makassar"),    # UTC+8
        "WIT": ZoneInfo("Asia/Jayapura"),     # UTC+9
    }
    target_zone = ZoneInfo(target_tz)

    # Nama hari Indonesia (untuk dibuang)
    day_names = r"(Senin|Selasa|Rabu|Kamis|Jumat|Sabtu|Minggu)"
    day_pattern = re.compile(rf"^\s*{day_names}\s*,?\s*", flags=re.IGNORECASE)

    # Peta bulan Indonesia -> Inggris (agar parser universal)
    month_map = {
        # panjang
        "januari": "January", "februari": "February", "maret": "March",
        "april": "April", "mei": "May", "juni": "June", "juli": "July",
        "agustus": "August", "september": "September", "oktober": "October",
        "november": "November", "desember": "December",
        # pendek
        "jan": "Jan", "feb": "Feb", "mar": "Mar", "apr": "Apr",
        "mei": "May", "jun": "Jun", "jul": "Jul",
        "agu": "Aug", "sep": "Sep", "okt": "Oct", "nov": "Nov", "des": "Dec",
    }

    def replace_month_id_to_en(s: str) -> str:
        def _repl(match):
            w = match.group(0)
            key = w.lower()
            repl = month_map.get(key, w)
            # jaga kapitalisasi awal bila perlu
            return repl
        # ganti kata yang cocok dengan kunci di month_map sebagai token utuh
        pattern = re.compile(r"\b(" + "|".join(map(re.escape, month_map.keys())) + r")\b", flags=re.IGNORECASE)
        return pattern.sub(_repl, s)

    def preprocess(s: str):
        """Membersihkan string tanggal dan mendeteksi petunjuk timezone lokal (WIB/WITA/WIT)."""
        if s is None:
            return None, None
        if not isinstance(s, str):
            s = str(s)

        orig = s
        s = s.strip()

        # Deteksi penanda zona lokal (WIB/WITA/WIT) bila ada
        tz_hint = None
        for k in tz_map.keys():
            if re.search(rf"\b{k}\b", s, flags=re.IGNORECASE):
                tz_hint = k.upper()
                break

        # Buang nama hari di depan (mis: "Kamis, ")
        s = day_pattern.sub("", s)

        # Buang "WIB", "WITA", "WIT" & koma
        s = re.sub(r"\b(WIB|WITA|WIT)\b", "", s, flags=re.IGNORECASE)
        s = s.replace(",", " ")

        # Normalisasi spasi
        s = re.sub(r"\s+", " ", s).strip()

        # Ganti bulan Indonesia -> Inggris
        s = replace_month_id_to_en(s)

        return s, tz_hint

    def parse_any(s: str, tz_hint: str):
        """
        Strategi parsing berlapis:
        1) Coba pandas.to_datetime (ISO/umum)
        2) Coba dateutil.parser.parse(dayfirst=True, fuzzy=True)
        3) Jika hanya YYYY-MM-DD -> konstruksi manual
        """
        # Kasus kosong
        if not s:
            raise ValueError("String tanggal kosong setelah praproses.")

        # 1) pandas
        dt = pd.to_datetime(s, errors="coerce", utc=False)
        if pd.notna(dt):
            # pandas bisa return Timestamp (naive atau aware)
            if dt.tzinfo is None:
                # naive -> gunakan hint jika ada, else target_tz
                zone = tz_map.get(tz_hint, target_zone)
                dt = dt.to_pydatetime().replace(tzinfo=zone)
            else:
                dt = dt.tz_convert(target_zone)
            return dt

        # 2) dateutil (lebih permisif)
        try:
            d = du_parser.parse(s, dayfirst=True, fuzzy=True)
            if d.tzinfo is None:
                zone = tz_map.get(tz_hint, target_zone)
                d = d.replace(tzinfo=zone)
            else:
                d = d.astimezone(target_zone)
            return d
        except Exception as e:
            # 3) Format murni tanggal YYYY-MM-DD
            m = re.fullmatch(r"(\d{4})-(\d{2})-(\d{2})", s)
            if m:
                y, mo, d = map(int, m.groups())
                zone = tz_map.get(tz_hint, target_zone)
                return datetime(y, mo, d, 0, 0, 0, tzinfo=zone)
            raise e  # lempar balik kalau benar-benar tidak bisa

    std_list = []
    dt_list = []
    err_list = []

    for i, raw in enumerate(df[src_col].tolist()):
        try:
            cleaned, tz_hint = preprocess(raw)
            if cleaned is None:
                raise ValueError("Nilai None pada tanggal.")
            dt = parse_any(cleaned, tz_hint)
            # pastikan di target tz
            dt = dt.astimezone(target_zone)

            # string ISO standar (detik presisi)
            std = dt.isoformat(timespec="seconds")
            std_list.append(std)
            dt_list.append(dt)
            err_list.append(None)
        except Exception as e:
            logging.warning(f"Gagal parse index={i}, nilai='{raw}': {e}")
            std_list.append(None)
            dt_list.append(pd.NaT)
            err_list.append(str(e))

    df[output_col_str] = std_list
    df[output_col_dt] = dt_list
    df[error_col] = err_list

    # Info ringkas
    total = len(df)
    gagal = sum(x is not None for x in err_list)
    logging.info(f"Normalisasi selesai. Total baris: {total} | Berhasil: {total-gagal} | Gagal: {gagal}")

    return df

# ===========================
# PEMAKAIAN
# ===========================
# Pastikan df_all sudah ada di memori.
df_all = normalize_tanggal_berita(df_all)

2025-09-28 16:47:36,354 | WARNING | Gagal parse index=350, nilai='<NA>': String does not contain a date: <NA>
2025-09-28 16:47:36,355 | WARNING | Gagal parse index=358, nilai='<NA>': String does not contain a date: <NA>
2025-09-28 16:47:36,356 | WARNING | Gagal parse index=359, nilai='<NA>': String does not contain a date: <NA>
2025-09-28 16:47:36,356 | WARNING | Gagal parse index=360, nilai='<NA>': String does not contain a date: <NA>
2025-09-28 16:47:36,357 | WARNING | Gagal parse index=361, nilai='<NA>': String does not contain a date: <NA>
2025-09-28 16:47:36,357 | WARNING | Gagal parse index=362, nilai='<NA>': String does not contain a date: <NA>
2025-09-28 16:47:36,357 | WARNING | Gagal parse index=363, nilai='<NA>': String does not contain a date: <NA>
2025-09-28 16:47:36,358 | WARNING | Gagal parse index=364, nilai='<NA>': String does not contain a date: <NA>
2025-09-28 16:47:36,358 | WARNING | Gagal parse index=366, nilai='<NA>': String does not contain a date: <NA>
2025-09-28

In [7]:
# Sesuaikan output akhir hanya ke kolom yang diminta
# Gunakan tanggal hasil normalisasi string jika tersedia, jika tidak fallback ke kolom asli
if "tanggal_berita_std" in df_all.columns:
    df_all["tanggal_berita"] = df_all["tanggal_berita_std"].fillna(df_all.get("tanggal_berita"))

df_all = df_all[["judul_berita", "url_berita", "tanggal_berita"]]

## FILTER PANJANG JUDUL BERITA

In [8]:
# Filter junk data: buang judul yang kurang dari 40 karakter
before_count = len(df_all)
# pastikan tipe string dan tangani NaN
df_all["judul_berita"] = df_all["judul_berita"].astype("string")
mask = df_all["judul_berita"].fillna("").str.len() >= 40
removed = before_count - int(mask.sum())
df_all = df_all[mask].copy()
logging.info(f"Filter judul < 40 chars: dihapus {removed} baris | sisa: {len(df_all)}")

2025-09-28 16:47:36,487 | INFO | Filter judul < 40 chars: dihapus 218 baris | sisa: 357


## SIMPAN DAFTAR BERITA

In [9]:
cwd

'/Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita/monitoring-berita'

In [10]:
import json
# Filter df_all untuk hanya menampilkan data yang paling awal dari search date
config_path = os.path.join(cwd, 'config.json')

with open(config_path, "r", encoding="utf-8") as f:
  config = json.load(f)

min_date = config['search_date'][-1]

#df_all = df_all[df_all['tanggal_berita_dt'] >= min_date]

In [11]:
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Simpan ke akar workspace agar mudah ditemukan, atau tetap di folder daftar_berita
output_path = os.path.join(cwd, f"konsolidasi_berita/hasil_gabungan_{timestamp}.csv")

# Pastikan hanya kolom yang diminta
expected_cols = ["judul_berita", "url_berita", "tanggal_berita"]
missing = [c for c in expected_cols if c not in df_all.columns]
if missing:
    # Tambahkan kolom kosong jika ada yang kurang (untuk keamanan)
    for c in missing:
        df_all[c] = pd.NA
    df_all = df_all[expected_cols]
else:
    df_all = df_all[expected_cols]

# Simpan
df_all.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"File berhasil disimpan: {output_path}")

File berhasil disimpan: /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita/monitoring-berita/konsolidasi_berita/hasil_gabungan_20250928_164736.csv


In [12]:
import json
from pathlib import Path

# Baca config lama kalau ada
config_path_obj = Path(config_path)
if config_path_obj.exists():
    try:
        with open(config_path, "r", encoding="utf-8") as f:
            config = json.load(f)
    except Exception as e:
        logging.warning(f"Gagal membaca config.json: {e}")
        config = {}
else:
    config = {}

# Update dengan path terbaru
config["last_output_path"] = str(output_path)
config["last_konsolidasi_time"] = timestamp

# Simpan kembali
try:
    with open(config_path, "w", encoding="utf-8") as f:
        json.dump(config, f, ensure_ascii=False, indent=2)
    logging.info(f"Config berhasil diperbarui: {config_path}")
except Exception as e:
    logging.error(f"Gagal menyimpan config.json: {e}")

2025-09-28 16:47:36,529 | INFO | Config berhasil diperbarui: /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita/monitoring-berita/config.json
